# Processing cellpy batch - ica

### `{{cookiecutter.project_name}}::{{cookiecutter.session_id}}`

**Experimental-id:** `{{cookiecutter.notebook_name}}`  
**Short-name:** `{{cookiecutter.session_id}}`  
**Project:** `{{cookiecutter.project_name}}`  
**By:** `{{cookiecutter.author_name}}`   
**Date:** `{{cookiecutter.date}}`  

## Imports and setup

In [ ]:
import os
import pathlib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import holoviews as hv
from holoviews.core.io import Pickler, Unpickler
import hvplot.pandas
from holoviews import opts

import cellpy
from cellpy import prms
from cellpy import prmreader
from cellpy.utils import batch, helpers, plotutils, ica

In [ ]:
%matplotlib inline
hv.extension('bokeh')
pd.set_option('display.max_columns', 70)
print(f"cellpy version: {cellpy.__version__}")

In [ ]:
cellpy_batch_file = "cellpy_batch_" + "{{cookiecutter.session_id}}" + ".json"

In [ ]:
b = batch.from_journal(cellpy_batch_file)
b.link()
cell_labels = b.experiment.cell_names
print(" cell labels: ")
cell_labels

In [ ]:
# # Remove bad cycles (if you have saved it to the journal session)
# helpers.yank_outliers(b, remove_indexes=b.journal.session["bad_cycles"])

In [ ]:
# # It is usually nice to see the capacity plot in the same notebook.
# # You can use the b.plot_summary() method or plotutils.bplot e.g.
# plotutils.bplot(b, columns=["charge_capacity"], max_cycle=100)

## Plotting

This will output an interactive ica plot for 100 first cycles for each cell.

In [ ]:
ica_curves = dict()
cycles = range(1, 101)
for label in b.experiment.cell_names:
    print(label)
    d = b.experiment.data[label]
    try:
        tidy_ica = ica.dqdv_frames(d, cycle=cycles, voltage_resolution=0.005)
    except:
        print(f"[{label} unsuccessfull]")
    else:
        ica_curve = hv.Curve(tidy_ica, kdims=['voltage', 'cycle'], vdims=['dq'], label="Incremental capacity plot").groupby("cycle").overlay().opts(show_legend=False)
        ica_curves[label] = ica_curve

In [ ]:
NdLayout_ica = hv.NdLayout(ica_curves, kdims='label').cols(3)

In [ ]:
palette = 'Spectral'
NdLayout_ica.opts(
    hv.opts.Curve(color=hv.Palette(palette), tools=['hover'], xlim=(0.05,0.8)),
    hv.opts.NdOverlay(shared_axes=False),
    hv.opts.NdLayout()
)

## Tweaking dQ/dV parameters

In [ ]:
cell_id = b.experiment.cell_names[0]
c = b.experiment.data[cell_id]
cycle_number = 4

In [ ]:
cycle = c.get_cap(cycle=cycle_number, categorical_column=True, method="forth-and-forth", insert_nan=False)

In [ ]:
voltage_1, incremental_1 = ica.dqdv_cycle(cycle)
curve1 = hv.Curve((voltage_1, incremental_1), label="one").opts(width=800, xlabel="Voltage", ylabel="dqdv")
curve1

In [ ]:
voltage_2, incremental_2 = ica.dqdv_cycle(cycle, voltage_resolution=0.01)
curve2 = hv.Curve((voltage_2, incremental_2), label="two").opts(width=800, xlabel="Voltage", ylabel="dqdv")
curve1 * curve2

## Making and saving dQ/dV to html and pickle for later use

In [ ]:
cell_id = b.experiment.cell_names[0]
c = b.experiment.data[cell_id]

In [ ]:
cycle = [1,2, 10, 20]
tidy_ica = ica.dqdv_frames(c, cycle=cycles, voltage_resolution=0.005, normalizing_factor=1)
curves = hv.Curve(tidy_ica, kdims=['voltage', 'cycle'], vdims=['dq']).groupby("cycle").overlay()
# setting options using the hv.opts API

curves.opts(
    hv.opts.Curve(
        color=hv.Cycle('Category20'),
    ),
    hv.opts.NdOverlay(
        title=f"Evolution [{cell_id}]",
        show_legend=True,
        legend_position="right",
        width=600,
        xlim=(0, 1),
        ylim=(-10, 5),
        ylabel="dQ/dV",
    ),
)

In [ ]:
fig_label = f"{{cookiecutter.session_id}}_{cell_id}_ica_evolution"

# export to html:
hv.save(
    curves,
    f"out/{fig_label}.html",
    toolbar=True
)

# save as pickle (can be loaded in other notebooks)
Pickler.save(curves, f"out/{fig_label}.hvz",)

## Saving dQ/dV to csv files
This can be usefull for plotting with another plotting software

In [ ]:
# Saving the dQ/dV data for the three first cycles (formation) for all cells except the last
out = pathlib.Path("data/processed")
ica_curves = dict()
cycles = [1,2,3]
selected_cell_labels = cell_labels[:-1]  # skipping the last cell
for label in selected_cell_labels:
    print(label)
    d = b.experiment.data[label]
    try:
        wide_ica = ica.dqdv_frames(d, cycle=cycles, voltage_resolution=0.005, tidy=False)
    except:
        print(f"[{label} unsuccessfull]")
    else:
        fname = f"{label}_ica_formation.csv"
        wide_ica.to_csv(out / fname, sep=";", index=False)
        print(f" -> saved to {out / fname}")
        

## Links

### Notebooks
- notes and information [link](00_{{cookiecutter.notebook_name}}_notes.ipynb)
- processing raw data [link](01_{{cookiecutter.notebook_name}}_loader.ipynb)
- life [link](02_{{cookiecutter.notebook_name}}_life.ipynb)
- cycles [link](03_{{cookiecutter.notebook_name}}_cycles.ipynb)
- ica [link](04_{{cookiecutter.notebook_name}}_ica.ipynb)
- plots [link](05_{{cookiecutter.notebook_name}}_plots.ipynb)